In [1]:
import os
import sys

sys.path.insert(0, "/Users/czhang/Desktop/pacific-observatory/")
import pandas as pd
from bokeh.layouts import Row, column, gridplot
from bokeh.models import (Title, Legend, ColumnDataSource, Select, HoverTool,
                          BoxZoomTool, ResetTool, DataTable, DateFormatter,
                          TableColumn)
from bokeh.models.layouts import TabPanel, Tabs
from bokeh.plotting import figure, output_file, show, output_notebook



output_file(filename="epu_pic.html")

In [2]:
countries = [
    "pacific", "papua_new_guinea", "solomon_islands", "fiji", "samoa",
    "vanuatu"
]

tabs = []

for country in countries:
    epu_file = sys.path[0] + f"outputs/text/{country}/epu/{country}_epu.csv"
    epu = pd.read_csv(epu_file).drop("Unnamed: 0", axis=1)
    epu["date"] = pd.to_datetime(epu["date"], format="mixed")

    source = ColumnDataSource(epu)

    hover = HoverTool(tooltips=[('Date', '@date{%Y-%m}'),
                                ('EPU Index', '@epu_unweighted')],
                      formatters={'@date': 'datetime'})

    p = figure(height=400,
               width=700,
               x_axis_type="datetime",
               tools=[hover, BoxZoomTool(), ResetTool()])
    p.line("date", "epu_weighted", source=source, name="epu_weighted")

    # Add a vertical
    p.vspan(x=pd.to_datetime("2021-01-01"),
            line_dash="dashed",
            line_color="green",
            name="cutoff")

    epu_dt = (epu.set_index("date").groupby(
        [pd.Grouper(freq="Y")])[["epu_weighted",
                                 "epu_unweighted"]].mean().reset_index())

    dt_source = ColumnDataSource(epu_dt)

    columns = [
        TableColumn(field="date", title="Date", formatter=DateFormatter()),
        TableColumn(field="epu_weighted", title="EPU Weighted Index (Avg.)"),
        TableColumn(field="epu_unweighted",
                    title="EPU Unweighted Index (Avg.)"),
    ]
    dt = DataTable(source=dt_source, columns=columns, width=700, height=300)
    combined = column(p, dt)

    # Uppercase the first letter of the country name
    title = " ".join(w[0].upper() + w[1:] for w in country.split("_"))
    tab = TabPanel(child=combined, title=title)
    tabs.append(tab)

show(Tabs(tabs=tabs))
